In [1]:
from datasets.TrafficTmp import TrafficForecastingDataset
import torch
import numpy as np

rng = np.random.default_rng(seed=42)

dataset = TrafficForecastingDataset(
    root = './data',
    name = 'metrla2',
    num_samples=lambda data_len: data_len // 3,
    min_sample_distance=1,
    rng=rng
)

ids = np.arange(len(dataset))

# TR-TS split
tmp = int(len(ids) * 0.1)
ts_ids = ids[-tmp:]
tr_ids = ids[:-tmp]
# TR-VL split
tmp = int(len(tr_ids) * 0.1)
vl_ids = tr_ids[-tmp:]
tr_ids = tr_ids[:-tmp]

train_dataset = dataset[tr_ids]
valid_dataset = dataset[vl_ids]
test_dataset  = dataset[ts_ids] 

all_test_x = torch.stack([data.x for data in test_dataset], dim=0)  # (ts_ids, num_nodes, 1)
lb_base = all_test_x.diff(dim=0).abs().mean()   # Computes mean of |x[i+1] - x[i]| over time dimension

print(lb_base)

tensor(2.8342)


In [2]:
from train_and_eval import eval_model
from models.baseline.baseline import LB_ODE

eval_model(
    model=LB_ODE(model_path='./saved_models_optuna/lb_base'),
    valid_data=test_dataset,
    criterion=torch.nn.L1Loss(),
    scaler = None
)

/home/rcappi/.conda/envs/my_env/lib/python3.12/site-packages/juliacall/__init__.py:61: UserWarning: torch was imported before juliacall. This may cause a segfault. To avoid this, import juliacall before importing torch. For updates, see https://github.com/pytorch/pytorch/issues/78829.
  warnings.warn(


Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


2.8345751762390137